In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import warnings
import math
import sys
import holidays
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL,xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn import metrics

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from math import sqrt






In [8]:
#Reading the data from train and destination csv
dataframe = pd.read_csv('../train.csv', sep=',', nrows=100000)
destinations = pd.read_csv('../destinations.csv', sep=',')
dataframe.shape

(100000, 24)

In [9]:
#Renaming columns hotel_cluster to item id and isbooking to rating
dataframe = dataframe.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})

In [16]:
#describing the data
dataframe.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date_time                  100000 non-null  object 
 1   site_name                  100000 non-null  int64  
 2   posa_continent             100000 non-null  int64  
 3   user_location_country      100000 non-null  int64  
 4   user_location_region       100000 non-null  int64  
 5   user_location_city         100000 non-null  int64  
 6   orig_destination_distance  63078 non-null   float64
 7   user_id                    100000 non-null  int64  
 8   is_mobile                  100000 non-null  int64  
 9   is_package                 100000 non-null  int64  
 10  channel                    100000 non-null  int64  
 11  srch_ci                    99929 non-null   object 
 12  srch_co                    99929 non-null   object 
 13  srch_adults_cnt            100

# Feature Engineering


In [17]:
dataframe= dataframe.drop(['orig_destination_distance'],axis=1)
dataframe = dataframe.dropna()

In [19]:
#Getting USA holidays and Canada holidays
ca_holidays = holidays.Canada()
us_holidays = holidays.UnitedStates()

uk_holidays = holidays.UnitedKingdom()
gr_holidays = holidays.Germany()

In [20]:
# checking if check-in data or checkout date is a holiday in different countries
#Adding a column north_am_ci giving it a value of 1 if the check in date is a holiday
dataframe['north_am_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
#Adding a column north_am_co giving it a value of 1 if the check in date is a holiday
dataframe['north_am_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
#Adding a column europe_ci giving it a value of 1 if the check in date is a holiday
dataframe['europe_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)
#Adding a europe_co giving it a value of 1 if the check in date is a holiday
dataframe['europe_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)

In [1]:
dataframe.head(n=2)

NameError: name 'dataframe' is not defined

In [24]:
# extracting month from date_time
dataframe['click_month'] = pd.DatetimeIndex(dataframe['date_time']).month
dataframe['checkin_month'] = pd.DatetimeIndex(dataframe['srch_ci']).month
dataframe['checkout_month'] = pd.DatetimeIndex(dataframe['srch_co']).month

In [25]:
#Removing date-time and checkin and check out columns from datafram
dataframe= dataframe.drop(['date_time'],axis=1)
dataframe= dataframe.drop(['srch_ci'],axis=1)
dataframe= dataframe.drop(['srch_co'],axis=1)

In [26]:
dataframe.shape

(99929, 27)

In [27]:
dataframe.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile',
       'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'rating', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'item_id', 'north_am_ci', 'north_am_co', 'europe_ci', 'europe_co',
       'click_month', 'checkin_month', 'checkout_month'],
      dtype='object')

In [28]:
# categ_sparse / conti_dense
sparse_features = ['site_name','posa_continent','user_location_country','user_location_region','user_location_city',
             'user_id','is_mobile','is_package','channel','click_month','checkin_month','checkout_month',
            'srch_adults_cnt','srch_children_cnt','srch_rm_cnt','srch_destination_id','hotel_continent',
               'hotel_country','cnt','north_am_ci', 'north_am_co', 'europe_ci', 'europe_co','click_month',
                   'checkin_month','checkout_month']

dense_features = ['hotel_market']
target = ['rating']

## Label Encoding for Sparse Features and Normalization For Dense Features


In [29]:
# Performing Label encoding for the sparse features
for feature in sparse_features:
    lbe = LabelEncoder()
    dataframe[feature] = lbe.fit_transform(dataframe[feature])

In [30]:
# Performing min-max scaling for the dense features
mms = MinMaxScaler(feature_range=(0,1))
dataframe[dense_features] = mms.fit_transform(dataframe[dense_features])

### Generate feature columns
For sparse features, we transform them into dense vectors by embedding techniques. For dense numerical features, we concatenate them to the input tensors of fully connected layer.

In [31]:
# count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, dataframe[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:
#Split the model into train and test dataset
train, test = train_test_split(dataframe, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

### MOdel before Hyper paramter Tuning

In [37]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 22s - loss: 0.2733 - binary_crossentropy: 0.2731 - val_loss: 0.2487 - val_binary_crossentropy: 0.2483
Epoch 2/10
63954/63954 - 18s - loss: 0.2229 - binary_crossentropy: 0.2221 - val_loss: 0.2652 - val_binary_crossentropy: 0.2641
Epoch 3/10
63954/63954 - 19s - loss: 0.2078 - binary_crossentropy: 0.2065 - val_loss: 0.2876 - val_binary_crossentropy: 0.2859
Epoch 4/10
63954/63954 - 19s - loss: 0.1993 - binary_crossentropy: 0.1975 - val_loss: 0.2982 - val_binary_crossentropy: 0.2961
Epoch 5/10
63954/63954 - 18s - loss: 0.1927 - binary_crossentropy: 0.1905 - val_loss: 0.3111 - val_binary_crossentropy: 0.3086
Epoch 6/10
63954/63954 - 19s - loss: 0.1885 - binary_crossentropy: 

In [38]:
auc = roc_auc_score(test[target].values, pred_ans)

print("Root-mean-square:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3))

RMSE:	0.294000
MAE:	0.146000
MSE:	0.087000
AUC:	0.705000


### To get best parameters 


In [39]:
#Assume certain parameters in a dictionary to perform hyper parameter tuning to get the best parameters
param_rand = {'dnn_hidden_units' : [(1,1),(2,2),(4,4),(8,8),(29,29),(128,128),(256,256)],
              'cin_layer_size':[(1,1),(2,2),(4,4),(8,8),(32,32),(128,128)],
              'cin_split_half':[True,False],
              'dnn_use_bn':[True,False],
              'l2_reg_cin':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_linear':[1e-5,1e-3,1e-1,1,10],
              'l2_reg_embedding':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_dnn':[0,0.2,2,4],
              'dnn_dropout':[0, 0.3, 0.5, 0.7, 0.9]
             }
result = {}

### To find the best parameter for DNN_Hidden_Unit

In [41]:
dnn_hidden_units_dict = {}
for i in param_rand['dnn_hidden_units']:
    model = xDeepFM(linear_feature_columns,dnn_feature_columns, dnn_hidden_units=i, 
              seed=1024,task='binary')
    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    dnn_hidden_units_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['dnn_hidden_units']=dnn_hidden_units_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 22s - loss: 0.0813 - mean_squared_error: 0.0811 - val_loss: 0.0709 - val_mean_squared_error: 0.0705
Epoch 2/10
63954/63954 - 19s - loss: 0.0666 - mean_squared_error: 0.0660 - val_loss: 0.0725 - val_mean_squared_error: 0.0717
Epoch 3/10
63954/63954 - 19s - loss: 0.0633 - mean_squared_error: 0.0624 - val_loss: 0.0771 - val_mean_squared_error: 0.0761
Epoch 4/10
63954/63954 - 19s - loss: 0.0611 - mean_squared_error: 0.0599 - val_loss: 0.0790 - val_mean_squared_error: 0.0778
Epoch 5/10
63954/63954 - 19s - loss: 0.0594 - mean_squared_error: 0.0581 - val_loss: 0.0814 - val_mean_squared_error: 0.0801
Epoch 6/10
63954/63954 - 20s - loss: 0.0579 - mean_squared_error: 0.0565 - val_loss: 0.0808 - val_mean_squared_error: 0.0794
Epoch 7/10
63954/63954 - 20s - loss: 0.0567 - mean_squared_error: 0.0553 - val_loss: 0.0835 - val_mean_squared_error: 0.0821
Epoch 8/10
63954/63954 - 20s - loss: 0.0557 - mean_squared_error: 0.0543 - 

Epoch 4/10
63954/63954 - 26s - loss: 0.0607 - mean_squared_error: 0.0597 - val_loss: 0.0773 - val_mean_squared_error: 0.0763
Epoch 5/10
63954/63954 - 26s - loss: 0.0591 - mean_squared_error: 0.0580 - val_loss: 0.0792 - val_mean_squared_error: 0.0781
Epoch 6/10
63954/63954 - 26s - loss: 0.0579 - mean_squared_error: 0.0567 - val_loss: 0.0838 - val_mean_squared_error: 0.0826
Epoch 7/10
63954/63954 - 25s - loss: 0.0569 - mean_squared_error: 0.0556 - val_loss: 0.0837 - val_mean_squared_error: 0.0824
Epoch 8/10
63954/63954 - 26s - loss: 0.0559 - mean_squared_error: 0.0545 - val_loss: 0.0863 - val_mean_squared_error: 0.0849
Epoch 9/10
63954/63954 - 26s - loss: 0.0548 - mean_squared_error: 0.0534 - val_loss: 0.0881 - val_mean_squared_error: 0.0867
Epoch 10/10
63954/63954 - 25s - loss: 0.0542 - mean_squared_error: 0.0528 - val_loss: 0.0912 - val_mean_squared_error: 0.0898


In [42]:
dnn_hidden_units_dict

{(1, 1): {'RMSE': 0.284, 'MAE': 0.13, 'MSE': 0.081, 'AUC': 0.743},
 (2, 2): {'RMSE': 0.285, 'MAE': 0.127, 'MSE': 0.081, 'AUC': 0.736},
 (4, 4): {'RMSE': 0.288, 'MAE': 0.13, 'MSE': 0.083, 'AUC': 0.743},
 (8, 8): {'RMSE': 0.287, 'MAE': 0.139, 'MSE': 0.083, 'AUC': 0.746},
 (29, 29): {'RMSE': 0.292, 'MAE': 0.143, 'MSE': 0.085, 'AUC': 0.742},
 (128, 128): {'RMSE': 0.278, 'MAE': 0.123, 'MSE': 0.077, 'AUC': 0.751},
 (256, 256): {'RMSE': 0.292, 'MAE': 0.141, 'MSE': 0.085, 'AUC': 0.748}}

### To get best parameter cin_layer_size

In [44]:
cin_layer_size_dict = {}
for i in param_rand['cin_layer_size']:
    model= xDeepFM(linear_feature_columns,dnn_feature_columns,cin_layer_size=i,dnn_hidden_units=(256, 256),
                   cin_activation='relu', seed=1024,task='binary',cin_split_half=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    cin_layer_size_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['cin_layer_size']=cin_layer_size_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 17s - loss: 0.0801 - mean_squared_error: 0.0800 - val_loss: 0.0710 - val_mean_squared_error: 0.0707
Epoch 2/10
63954/63954 - 12s - loss: 0.0662 - mean_squared_error: 0.0657 - val_loss: 0.0728 - val_mean_squared_error: 0.0722
Epoch 3/10
63954/63954 - 12s - loss: 0.0629 - mean_squared_error: 0.0622 - val_loss: 0.0746 - val_mean_squared_error: 0.0738
Epoch 4/10
63954/63954 - 12s - loss: 0.0607 - mean_squared_error: 0.0598 - val_loss: 0.0783 - val_mean_squared_error: 0.0772
Epoch 5/10
63954/63954 - 12s - loss: 0.0592 - mean_squared_error: 0.0582 - val_loss: 0.0812 - val_mean_squared_error: 0.0801
Epoch 6/10
63954/63954 - 12s - loss: 0.0582 - mean_squared_error: 0.0570 - val_loss: 0.0801 - val_mean_squared_error: 0.0789
Epoch 7/10
63954/63954 - 12s - loss: 0.0571 - mean_squared_error: 0.0559 - val_loss: 0.0829 - val_mean_squared_error: 0.0815
Epoch 8/10
63954/63954 - 12s - loss: 0.0563 - mean_squared_error: 0.0549 - 

In [45]:
cin_layer_size_dict

{(1, 1): {'RMSE': 0.286, 'MAE': 0.125, 'MSE': 0.082, 'AUC': 0.749},
 (2, 2): {'RMSE': 0.284, 'MAE': 0.132, 'MSE': 0.081, 'AUC': 0.748},
 (4, 4): {'RMSE': 0.284, 'MAE': 0.125, 'MSE': 0.081, 'AUC': 0.748},
 (8, 8): {'RMSE': 0.281, 'MAE': 0.127, 'MSE': 0.079, 'AUC': 0.75},
 (32, 32): {'RMSE': 0.286, 'MAE': 0.126, 'MSE': 0.082, 'AUC': 0.746},
 (128, 128): {'RMSE': 0.283, 'MAE': 0.126, 'MSE': 0.08, 'AUC': 0.747}}

### To get Best CIN_split half


In [46]:
cin_split_half_dict = {}
for i in param_rand['cin_split_half']:
    model= xDeepFM(linear_feature_columns,dnn_feature_columns,dnn_hidden_units=(256, 256),
                    cin_layer_size=(128, 128),cin_activation='relu',cin_split_half=i,
                    seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    cin_split_half_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['cin_split_half']=cin_split_half_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 33s - loss: 0.0784 - mean_squared_error: 0.0782 - val_loss: 0.0722 - val_mean_squared_error: 0.0719
Epoch 2/10
63954/63954 - 28s - loss: 0.0661 - mean_squared_error: 0.0656 - val_loss: 0.0729 - val_mean_squared_error: 0.0722
Epoch 3/10
63954/63954 - 28s - loss: 0.0629 - mean_squared_error: 0.0621 - val_loss: 0.0751 - val_mean_squared_error: 0.0742
Epoch 4/10
63954/63954 - 29s - loss: 0.0605 - mean_squared_error: 0.0596 - val_loss: 0.0777 - val_mean_squared_error: 0.0767
Epoch 5/10
63954/63954 - 28s - loss: 0.0590 - mean_squared_error: 0.0580 - val_loss: 0.0797 - val_mean_squared_error: 0.0786
Epoch 6/10
63954/63954 - 31s - loss: 0.0579 - mean_squared_error: 0.0567 - val_loss: 0.0826 - val_mean_squared_error: 0.0814
Epoch 7/10
63954/63954 - 29s - loss: 0.0569 - mean_squared_error: 0.0557 - val_loss: 0.0846 - val_mean_squared_error: 0.0833
Epoch 8/10
63954/63954 - 28s - loss: 0.0559 - mean_squared_error: 0.0546 - 

In [47]:
cin_split_half_dict

{True: {'RMSE': 0.285, 'MAE': 0.119, 'MSE': 0.082, 'AUC': 0.75},
 False: {'RMSE': 0.289, 'MAE': 0.135, 'MSE': 0.083, 'AUC': 0.746}}

### TO get best paramter l2_reg_CIN

In [49]:
l2_reg_cin_dict={}
for i in param_rand['l2_reg_cin']:
    model = xDeepFM(linear_feature_columns,dnn_feature_columns,dnn_hidden_units=(256, 256),
                    cin_layer_size=(128, 128),cin_activation='relu',cin_split_half=True,l2_reg_cin=i,
                    seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_cin_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
    
result['l2_reg_cin']=l2_reg_cin_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 40s - loss: 0.0786 - mean_squared_error: 0.0784 - val_loss: 0.0708 - val_mean_squared_error: 0.0705
Epoch 2/10
63954/63954 - 31s - loss: 0.0663 - mean_squared_error: 0.0658 - val_loss: 0.0730 - val_mean_squared_error: 0.0724
Epoch 3/10
63954/63954 - 30s - loss: 0.0629 - mean_squared_error: 0.0621 - val_loss: 0.0763 - val_mean_squared_error: 0.0755
Epoch 4/10
63954/63954 - 29s - loss: 0.0607 - mean_squared_error: 0.0597 - val_loss: 0.0775 - val_mean_squared_error: 0.0765
Epoch 5/10
63954/63954 - 29s - loss: 0.0591 - mean_squared_error: 0.0580 - val_loss: 0.0801 - val_mean_squared_error: 0.0790
Epoch 6/10
63954/63954 - 29s - loss: 0.0580 - mean_squared_error: 0.0568 - val_loss: 0.0813 - val_mean_squared_error: 0.0800
Epoch 7/10
63954/63954 - 29s - loss: 0.0571 - mean_squared_error: 0.0558 - val_loss: 0.0814 - val_mean_squared_error: 0.0800
Epoch 8/10
63954/63954 - 29s - loss: 0.0561 - mean_squared_error: 0.0548 - 

In [50]:
l2_reg_cin_dict

{1e-07: {'RMSE': 0.285, 'MAE': 0.13, 'MSE': 0.081, 'AUC': 0.748},
 1e-05: {'RMSE': 0.279, 'MAE': 0.13, 'MSE': 0.078, 'AUC': 0.75},
 0.001: {'RMSE': 0.29, 'MAE': 0.142, 'MSE': 0.084, 'AUC': 0.746},
 0.1: {'RMSE': 0.286, 'MAE': 0.128, 'MSE': 0.082, 'AUC': 0.745},
 1: {'RMSE': 0.285, 'MAE': 0.129, 'MSE': 0.081, 'AUC': 0.748}}

### To get best paramter l2_reg_linear

In [51]:
l2_reg_linear_dict={}
for i in param_rand['l2_reg_linear']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), l2_reg_linear=i,
                    cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
                    dnn_use_bn=False,seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_linear_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
    
result['l2_reg_linear']=l2_reg_linear_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 48s - loss: 0.0784 - mean_squared_error: 0.0782 - val_loss: 0.0709 - val_mean_squared_error: 0.0705
Epoch 2/10
63954/63954 - 50s - loss: 0.0661 - mean_squared_error: 0.0655 - val_loss: 0.0729 - val_mean_squared_error: 0.0723
Epoch 3/10
63954/63954 - 77s - loss: 0.0627 - mean_squared_error: 0.0619 - val_loss: 0.0761 - val_mean_squared_error: 0.0752
Epoch 4/10
63954/63954 - 77s - loss: 0.0607 - mean_squared_error: 0.0597 - val_loss: 0.0770 - val_mean_squared_error: 0.0760
Epoch 5/10
63954/63954 - 77s - loss: 0.0592 - mean_squared_error: 0.0582 - val_loss: 0.0794 - val_mean_squared_error: 0.0783
Epoch 6/10
63954/63954 - 77s - loss: 0.0580 - mean_squared_error: 0.0568 - val_loss: 0.0814 - val_mean_squared_error: 0.0802
Epoch 7/10
63954/63954 - 77s - loss: 0.0570 - mean_squared_error: 0.0558 - val_loss: 0.0855 - val_mean_squared_error: 0.0841
Epoch 8/10
63954/63954 - 76s - loss: 0.0563 - mean_squared_error: 0.0550 - 

In [52]:
l2_reg_linear_dict

{1e-05: {'RMSE': 0.287, 'MAE': 0.134, 'MSE': 0.082, 'AUC': 0.748},
 0.001: {'RMSE': 0.288, 'MAE': 0.137, 'MSE': 0.083, 'AUC': 0.75},
 0.1: {'RMSE': 0.287, 'MAE': 0.128, 'MSE': 0.082, 'AUC': 0.75},
 1: {'RMSE': 0.287, 'MAE': 0.137, 'MSE': 0.082, 'AUC': 0.749},
 10: {'RMSE': 0.284, 'MAE': 0.128, 'MSE': 0.081, 'AUC': 0.751}}

### To get best paramter of l2_reg_embedding


In [53]:
l2_reg_embedding_dict={}
for i in param_rand['l2_reg_embedding']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns,dnn_hidden_units=(256, 256), l2_reg_linear=0.1,
            l2_reg_embedding=i,cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', 
            l2_reg_cin=1e-07, seed=1024, task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    l2_reg_embedding_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['l2_reg_embedding']=l2_reg_embedding_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 57s - loss: 0.0795 - mean_squared_error: 0.0786 - val_loss: 0.0709 - val_mean_squared_error: 0.0707
Epoch 2/10
63954/63954 - 43s - loss: 0.0654 - mean_squared_error: 0.0652 - val_loss: 0.0727 - val_mean_squared_error: 0.0725
Epoch 3/10
63954/63954 - 43s - loss: 0.0617 - mean_squared_error: 0.0615 - val_loss: 0.0754 - val_mean_squared_error: 0.0752
Epoch 4/10
63954/63954 - 42s - loss: 0.0594 - mean_squared_error: 0.0592 - val_loss: 0.0764 - val_mean_squared_error: 0.0762
Epoch 5/10
63954/63954 - 45s - loss: 0.0577 - mean_squared_error: 0.0575 - val_loss: 0.0792 - val_mean_squared_error: 0.0790
Epoch 6/10
63954/63954 - 45s - loss: 0.0564 - mean_squared_error: 0.0562 - val_loss: 0.0805 - val_mean_squared_error: 0.0803
Epoch 7/10
63954/63954 - 46s - loss: 0.0550 - mean_squared_error: 0.0548 - val_loss: 0.0814 - val_mean_squared_error: 0.0812
Epoch 8/10
63954/63954 - 45s - loss: 0.0541 - mean_squared_error: 0.0539 - 

In [54]:
l2_reg_embedding_dict

{1e-07: {'RMSE': 0.285, 'MAE': 0.134, 'MSE': 0.081, 'AUC': 0.739},
 1e-05: {'RMSE': 0.283, 'MAE': 0.134, 'MSE': 0.08, 'AUC': 0.75},
 0.001: {'RMSE': 0.264, 'MAE': 0.124, 'MSE': 0.069, 'AUC': 0.772},
 0.1: {'RMSE': 0.262, 'MAE': 0.141, 'MSE': 0.068, 'AUC': 0.752},
 1: {'RMSE': 0.265, 'MAE': 0.145, 'MSE': 0.07, 'AUC': 0.72}}

### To get best parameter l2-reg-dnn

In [ ]:
l2_reg_dnn_dict={}
for i in param_rand['l2_reg_dnn']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), l2_reg_linear=0.1,
            cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
            l2_reg_embedding=1e-05, l2_reg_dnn=i,seed=1024,task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_dnn_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['l2_reg_dnn']=l2_reg_dnn_dict

### To get Best parameter dnn


In [ ]:
dnn_dropout_dict={}
for i in param_rand['dnn_dropout']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),
                   cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
                   seed=1024, dnn_dropout=i, dnn_activation='relu',task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    dnn_dropout_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['dnn_dropout']=dnn_dropout_dict

### To train the model with best hyperparamter


In [ ]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns, dnn_hidden_units=(128,128), 
             init_std=0.0001, seed=1024,task='binary')

model.compile("adam", "mse", metrics=['mse'])

In [ ]:
history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)

auc = roc_auc_score(test[target].values, pred_ans)

In [ ]:
print("RMSE:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3))